In [11]:
import pandas as pd 

In [4]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ============================================
# CONFIGURAÇÕES INICIAIS
# ============================================
INPUT_PATH = "C:/Users/Administrator/Desktop/FlavioWork/Dirtyfile/vendas_modificado.csv"
OUTPUT_PATH = "C:/Users/Administrator/Desktop/FlavioWork/Dirtyfile/vendas_limpo.csv"
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# ============================================
# 1. CARREGAMENTO E RENOMEAÇÃO DAS COLUNAS
# ============================================
try:
    df = pd.read_csv(INPUT_PATH, encoding='utf-8')

    # Renomear colunas com base no padrão fornecido (em maiúsculas)
    colunas_padronizadas = [
        "ID_DA_COMPRA", "DATA", "HORA", "CLIENTE", "PRODUTO", "VALOR", "QUANTIDADE",
        "TOTAL", "STATUS", "CIDADE", "ESTADO", "PAIS", "CEP", "FRETE", "PAGAMENTO",
        "VENDEDOR", "MARCA"
    ]
    df.columns = colunas_padronizadas

    original_stats = {
        'total_registros': len(df),
        'colunas': df.columns.tolist(),
        'missing_values': df.isnull().sum().to_dict(),
        'duplicatas': df.duplicated().sum()
    }
except Exception as e:
    raise Exception(f"Erro ao carregar arquivo: {str(e)}")

# ============================================
# 2. TRATAMENTO DE DADOS FALTANTES
# ============================================
def fill_missing_values(df):
    numeric_cols = ['VALOR', 'QUANTIDADE', 'FRETE', 'TOTAL']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df[numeric_cols] = df[numeric_cols].fillna({
        'VALOR': 0,
        'QUANTIDADE': 1,
        'FRETE': 0,
        'TOTAL': 0
    })

    text_cols = ['CLIENTE', 'PRODUTO', 'STATUS', 'CIDADE', 'ESTADO',
                 'PAIS', 'PAGAMENTO', 'VENDEDOR', 'MARCA']
    df[text_cols] = df[text_cols].fillna('desconhecido').apply(lambda x: x.str.strip())

    df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce').fillna(pd.Timestamp('2000-01-01'))
    df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S', errors='coerce')
    df['HORA'] = df['HORA'].dt.strftime('%H:%M:%S').fillna('00:00:00')

    return df

df = fill_missing_values(df)

# ============================================
# 3. PADRONIZAÇÃO DE DADOS
# ============================================
def standardize_data(df):
    text_cols = ['CLIENTE', 'PRODUTO', 'STATUS', 'CIDADE', 'ESTADO',
                 'PAIS', 'PAGAMENTO', 'VENDEDOR', 'MARCA']
    df[text_cols] = df[text_cols].apply(lambda x: x.str.lower().str.strip())

    product_mapping = {
        r'.*iphone.*': 'iphone',
        r'.*samsung.*galaxy.*': 'samsung galaxy',
        r'.*tv.*led.*': 'tv led',
        r'.*laptop.*|.*notebook.*': 'notebook',
        r'.*macbook.*': 'macbook',
        r'.*tablet.*': 'tablet'
    }

    for pattern, replacement in product_mapping.items():
        df['PRODUTO'] = df['PRODUTO'].str.replace(pattern, replacement, regex=True)

    df['CEP'] = df['CEP'].astype(str)
    df['CEP'] = df['CEP'].str.replace(r'\D', '', regex=True)
    df['CEP'] = df['CEP'].str.pad(8, fillchar='0')
    invalid_ceps = ~df['CEP'].str.match(r'^\d{8}$', na=False)
    df.loc[invalid_ceps, 'CEP'] = '00000000'

    return df

df = standardize_data(df)

# ============================================
# 4. VALIDAÇÃO E CORREÇÃO DE DADOS
# ============================================
def validate_data(df):
    df['TOTAL_CALCULADO'] = df['VALOR'] * df['QUANTIDADE'] + df['FRETE']
    df['DIF_TOTAL'] = abs(df['TOTAL'] - df['TOTAL_CALCULADO'])
    df['TOTAL'] = np.where(df['DIF_TOTAL'] > 0.01, df['TOTAL_CALCULADO'], df['TOTAL'])
    df.drop(columns=['TOTAL_CALCULADO', 'DIF_TOTAL'], inplace=True)

    df = df[(df['QUANTIDADE'] > 0) & (df['QUANTIDADE'] <= 1000)]
    df = df[(df['VALOR'] >= 0) & (df['VALOR'] <= 100000)]
    df = df[(df['FRETE'] >= 0) & (df['FRETE'] <= 5000)]

    brazilian_states = [
        'ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma',
        'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn',
        'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to'
    ]
    df = df[df['ESTADO'].isin(brazilian_states + ['desconhecido'])]

    return df

df = validate_data(df)

# ============================================
# 5. REMOÇÃO DE DUPLICATAS E LIMPEZA FINAL
# ============================================
df = df.drop_duplicates()
df = df.reset_index(drop=True)

# ============================================
# 6. EXPORTAÇÃO E RELATÓRIO
# ============================================
try:
    df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')

    cleaning_report = f"""
    ==============================
    RELATÓRIO DE LIMPEZA DE DADOS
    ==============================

    DADOS ORIGINAIS:
    - Total de registros: {original_stats['total_registros']}
    - Colunas: {', '.join(original_stats['colunas'])}
    - Valores faltantes: {sum(original_stats['missing_values'].values())}
    - Registros duplicados: {original_stats['duplicatas']}

    TRANSFORMAÇÕES APLICADAS:
    1. Tratamento de valores faltantes
    2. Padronização de dados
    3. Validação e correção
    4. Remoção de duplicatas

    RESULTADO FINAL:
    - Total de registros válidos: {len(df)}
    - Registros removidos: {original_stats['total_registros'] - len(df)}
    - Percentual mantido: {(len(df) / original_stats['total_registros']) * 100:.2f}%

    Arquivo salvo: {OUTPUT_PATH}
    Data do processo: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    """

    print(cleaning_report)

except Exception as e:
    raise Exception(f"Erro ao salvar arquivo limpo: {str(e)}")



    RELATÓRIO DE LIMPEZA DE DADOS

    DADOS ORIGINAIS:
    - Total de registros: 368752
    - Colunas: ID_DA_COMPRA, DATA, HORA, CLIENTE, PRODUTO, VALOR, QUANTIDADE, TOTAL, STATUS, CIDADE, ESTADO, PAIS, CEP, FRETE, PAGAMENTO, VENDEDOR, MARCA
    - Valores faltantes: 14736
    - Registros duplicados: 5158

    TRANSFORMAÇÕES APLICADAS:
    1. Tratamento de valores faltantes
    2. Padronização de dados
    3. Validação e correção
    4. Remoção de duplicatas

    RESULTADO FINAL:
    - Total de registros válidos: 354348
    - Registros removidos: 14404
    - Percentual mantido: 96.09%

    Arquivo salvo: C:/Users/Administrator/Desktop/FlavioWork/Dirtyfile/vendas_limpo.csv
    Data do processo: 2025-04-21 17:58:39
    
